In [1]:
import sys
import pandas as pd
import numpy as np
from numpy import absolute, mean, sort, std

import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)

from sklearn import datasets, metrics, preprocessing, model_selection
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split, KFold,RepeatedKFold, cross_val_score, cross_validate, cross_val_predict, GridSearchCV, RandomizedSearchCV, validation_curve, learning_curve
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, mean_absolute_error, max_error

import skopt
from skopt import BayesSearchCV 

from missingpy import MissForest

import shap
from BorutaShap import BorutaShap

import xgboost
import lightgbm
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.svm import SVR

import warnings
warnings.filterwarnings("ignore")

seed = 0

dataset = pd.read_csv("training_data.txt", sep="\t")
dataset = dataset.rename({'IPA_BP':'IPA_BP_annotation'}, axis=1)

data = dataset.drop(
    ["Gene", ], 1
)  

data["BPlabel_encoded"] = data["BPlabel"].map(
    {"most likely": 1, "probable": 0.65, "least likely": 0.1}
)
Y = data["BPlabel_encoded"]

natest = data.isnull().sum()
natest.sort_values(inplace=True)

percent_missing = data.isnull().sum() * 100 / len(data)
missing_value_df = pd.DataFrame(
    {"column_name": data.columns, "percent_missing": percent_missing}
)
missing_value_df.sort_values("percent_missing", inplace=True)

natest = natest.to_frame()
missingdata = natest.join(missing_value_df)

missingdata.to_csv("training_data_missingness.csv")

data_drop = data.drop(
    ["BPlabel", "BPlabel_encoded",], 1
) 

null_counts = data_drop.isnull().sum() / len(data_drop)

selection = missing_value_df[missing_value_df["percent_missing"] < 25.00]
list(selection["column_name"])

dat = data[list(selection["column_name"])]
dat["Gene"] = dataset["Gene"]

dt2 = dat
dat = dat.set_index("Gene")

df = dt2
df = df.set_index("Gene")
df["BPlabel_encoded"] = df["BPlabel"].map(
    {"most likely": 1, "probable": 0.65, "least likely": 0.1}
)

# removing remaining variant-level features strongly correlating with gene length:
df = df.drop(["BPlabel", "Gene_length", "CADD_RAW", "gwastrait",'ExomiserScore',
 'ppiscore_Exomiser','DNaseCluster_count', 'H3k4me1_count','CpGcount', 'EnhancerCount','H3k4me3_count', 'H3k27Ac_count'], 1, errors='ignore')

X = MinMaxScaler().fit_transform(df)
imputer = MissForest(random_state=seed)
X = pd.DataFrame(imputer.fit_transform(X), index=df.index, columns=df.columns)

Xcor = X
Xcor = pd.DataFrame(data=Xcor, columns=X.columns)
corr = Xcor.corr(method="spearman").abs()
upper = corr.where(np.triu(np.ones(corr.shape),k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]
print("Dropped features with > 0.9 correlation:", to_drop)

selected_columns = X.drop(X[to_drop], axis=1)

dt_towrite = data[list(selected_columns)]
dt_towrite["Gene"] = dataset["Gene"]
dt_towrite["BPlabel"] = dataset["BPlabel"]
dt_towrite = dt_towrite.drop(["BPlabel_encoded"], 1)

dt_towrite.to_csv("training_cleaned.csv", header=True, index=False)

data = pd.read_csv("training_cleaned.csv", header=0, sep=",")

data["BPlabel_encoded"] = data["BPlabel"].map(
    {"most likely": 1, "probable": 0.65, "least likely": 0.1}
)
Y = data["BPlabel_encoded"]
data = data.drop(["BPlabel"], 1)

df1 = data.iloc[:, :-1]
print('Training data dimensions with all features:', df1.shape)

df = df1.set_index("Gene")

imputer = MissForest(random_state=seed)
X = pd.DataFrame(imputer.fit_transform(df), index=df.index, columns=df.columns)

X.to_csv(r"imputed_training_data.csv", index=False)

data = pd.read_csv("training_cleaned.csv", header=0, sep=",")

data["BPlabel_encoded"] = data["BPlabel"].map(
    {"most likely": 1, "probable": 0.65, "least likely": 0.1}
)
Y = data["BPlabel_encoded"]
data = data.drop(["BPlabel"], 1)

X = pd.read_csv("imputed_training_data.csv", header=0)
X.columns = [
    regex.sub("_", col) if any(x in str(col) for x in set(("[", "]", "<"))) else col
    for col in X.columns.values
]

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=seed
)

xgbr = xgboost.XGBRegressor(random_state=seed, objective='reg:squarederror') 
xgbr_params = {
    'max_depth':  (1, 4), 
    'learning_rate': (0.01, 0.2, 'log-uniform'),  
    'n_estimators':  (10, 50), 
    'reg_alpha':  (1, 10, 'log-uniform'), 
    'reg_lambda':  (1, 10, 'log-uniform')} 

lgbm = LGBMRegressor(random_state=seed)
lgbm_params = {
    "max_depth": (1, 4),
    "learning_rate": (0.01, 0.2, "log-uniform"),
    "n_estimators": (10, 50),
    "reg_alpha": (1, 10, "log-uniform"),
    "reg_lambda": (1, 10, "log-uniform"),
}

catboost = CatBoostRegressor(random_seed=seed, verbose=False)
cat_params = {
     "iterations": (10, 50),
     'learning_rate': (0.01, 0.2, 'log-uniform'), 
     'depth':  (1, 4), 
}


gbr = GradientBoostingRegressor(random_state=seed)
gbr_params = {
    'learning_rate': (0.01, 0.2),
    'max_depth': (1, 4),
    "max_features":["log2","sqrt", "auto"],
    "criterion": ["friedman_mse", "mse"],
    'n_estimators': (10, 50)
    }

rfr = RandomForestRegressor(random_state=seed)
rfr_params={'n_estimators': (10, 50), 
             'max_features': ['auto', 'sqrt', 'log2'],
             'max_depth' : (1, 4),} 

dt = DecisionTreeRegressor(random_state=seed)
dt_params= {"criterion": ["mse", "mae"],
            'max_features': ['auto', 'sqrt', 'log2'],
            'max_depth' : (1, 4)}

extra = ExtraTreesRegressor(random_state=seed)
extra_params ={'n_estimators': (10, 50), 
             'max_features': ['auto', 'sqrt', 'log2'],
             'max_depth' : (1, 4),}

knr = KNeighborsRegressor()
knr_params = {
    'n_neighbors':[7,9,11,13,15,17],
    'weights' : ['uniform','distance'],
    'metric' : ['euclidean','manhattan']}


lasso = Lasso(random_state=seed)
lasso_params =  {"alpha": (0.001, 0.01, 0.1),
                "max_iter": (500, 1000, 5000),}

elastic = ElasticNet(random_state=seed, tol=1)
elastic_params = {
    "max_iter": (500, 1000, 5000),
    "alpha": (0.001, 0.01, 0.1),
    "l1_ratio": np.arange(0.0, 1.0)}

svr = SVR()
svr_params = {
    'kernel': ['rbf'],
   'C': (1e0, 1e3),
   'gamma': (1e-4, 1e-3)}

inner_cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=seed)
outer_cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=seed)

models = []

models.append(("LGBM", BayesSearchCV(lgbm, lgbm_params, cv=inner_cv, iid=False, n_jobs=1, random_state=seed)))
models.append(('XGBR', BayesSearchCV(xgbr, xgbr_params, cv=inner_cv,iid=False,n_jobs=1, random_state=seed))) 
models.append(("CB", BayesSearchCV(catboost, cat_params, cv=inner_cv, iid=False, n_jobs=1, random_state=seed)))
models.append(('GBR', BayesSearchCV(gbr, gbr_params, cv=inner_cv,iid=False, n_jobs=1, random_state=seed)))
models.append(('RFR', BayesSearchCV(rfr, rfr_params, cv=inner_cv,iid=False, n_jobs=1, random_state=seed)))
models.append(('DT', BayesSearchCV(dt, dt_params, cv=inner_cv, iid=False, n_jobs=1, random_state=seed)))
models.append(('ExtraTrees', BayesSearchCV(extra, extra_params, cv=inner_cv, iid=False, n_jobs=1, random_state=seed)))


results = []
names = []
medians =[]
scoring = ['r2', 'neg_mean_squared_error', 'max_error', 'neg_mean_absolute_error',
          'explained_variance','neg_root_mean_squared_error',
           'neg_median_absolute_error'] 

models_list_r2 = []
models_list_predr2 = []

def press_statistic(Y, y_pred2, xs):
    res = y_pred2 - Y
    hat = xs.dot(np.linalg.pinv(xs))
    den = 1 - np.diagonal(hat)
    sqr = np.square(res / den)
    return sqr.sum()


def predicted_r2(Y, y_pred2, xs):
    press = press_statistic(Y=Y, y_pred2=y_pred2, xs=xs)
    sst = np.square(Y - Y.mean()).sum()
    return 1 - press / sst


def r2(Y, y_pred2):
    sse = np.square(y_pred2 - Y).sum()
    sst = np.square(Y - Y.mean()).sum()
    return 1 - sse / sst


df3 = pd.DataFrame(data=X, columns=X.columns)
df3.columns = [
    regex.sub("_", col) if any(x in str(col) for x in set(("[", "]", "<"))) else col
    for col in X.columns.values
]
X_importance = X_test

for name, model in models:
    nested_cv_results = model_selection.cross_validate(model, X, Y, cv=outer_cv, scoring=scoring, error_score="raise")
    nested_cv_results2 = model_selection.cross_val_score(model, X, Y, cv=outer_cv, scoring='r2', error_score="raise")
    results.append(nested_cv_results2)
    names.append(name)
    medians.append(np.median(nested_cv_results['test_r2']))
    print(name, 'Nested CV results for all scores:', '\n', nested_cv_results, '\n')
    print(name, 'r2 Nested CV Median', np.median(nested_cv_results['test_r2']))
    print(name, 'MSE Nested CV Median', np.median(nested_cv_results['test_neg_mean_squared_error'] ))
    print(name, 'RMSE Nested CV Median', np.median(nested_cv_results['test_neg_root_mean_squared_error'] ))
    print(name, 'Explained Variance Nested CV Median', np.median(nested_cv_results['test_explained_variance'] ))
    print(name, 'MAE Nested CV Median', np.median(nested_cv_results['test_neg_mean_absolute_error'] ))
    model.fit(X, Y)
    print('\n')
    print("Best Parameters: \n{}\n".format(model.best_params_))
    print("Best Estimator:", model.best_estimator_)
    best_model = model.best_estimator_
    print('\n')
    print('Non-nested CV Results:')
    best_model.fit(X_train, Y_train)
    y_pred_train = best_model.predict(X_train)
    y_pred = best_model.predict(X_test)
    print(name, 'Train MSE:', mean_squared_error(Y_train, y_pred_train), 'Test MSE:', mean_squared_error(Y_test, y_pred))
    print(name, 'Train Explained Variance Score:', explained_variance_score(Y_train, y_pred_train), 'Test Explained Variance Score:', explained_variance_score(Y_test, y_pred))
    print(name, 'Train MAE:', mean_absolute_error(Y_train, y_pred_train),'Test MAE:', mean_absolute_error(Y_test, y_pred))
    print(name, 'Train Max Error:', max_error(Y_train, y_pred_train), 'Test Max Error:', max_error(Y_test, y_pred))
    print(name, 'Train r2:', r2_score(Y_train, y_pred_train), 'Test r2:', r2_score(Y_test, y_pred))
    print('\n')
    best_model.fit(X, Y)
    y_pred2 = best_model.predict(X)
    print(name, "Best model predicted r2:", predicted_r2(Y, y_pred2, X))
    #explainer = shap.TreeExplainer(best_model)
    #shap_values = explainer.shap_values(X)
    #X_importance = pd.DataFrame(data=X, columns=df3.columns)
    #print(name,'ALL FEATURES Ranked SHAP Importance:', X.columns[np.argsort(np.abs(shap_values).mean(0))[::-1]])
    #fig, ax = plt.subplots()
    #shap.summary_plot(shap_values, X)
    #fig.savefig("shap_summary_all_features" + name +".svg", format='svg', dpi=1200, bbox_inches = "tight")
    median_r2 = np.median(nested_cv_results['test_r2'])
    models_list_r2.append((best_model,  median_r2))
    predr2_score = predicted_r2(Y, y_pred2, X)
    models_list_predr2.append((best_model, predr2_score))

print('All r2 results:', results)         

best_model1, best_r2 = sorted(models_list_r2, key = lambda x: x[1], reverse=True)[0]
best_model2, best_pred_r2 = sorted(models_list_predr2, key = lambda x: x[1], reverse=True)[0]
print('Best model by median r2:',best_model1)
print('Best model by predicted r2:',best_model2)

Feature_Selector = BorutaShap(model=best_model1, importance_measure="shap", classification=False)

Feature_Selector.fit(X=X, y=Y, n_trials=200, random_state=seed)

subset = Feature_Selector.Subset()
X_boruta_sel = subset
X_boruta_sel

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Dropped features with > 0.9 correlation: ['Nerve - Tibial_GTExTPM', 'Heart - Left Ventricle_GTExTPM', 'Kidney - Medulla_GTExTPM', 'Lung_GTExTPM', 'Minor Salivary Gland_GTExTPM', 'Muscle - Skeletal_GTExTPM', 'Ovary_GTExTPM', 'Skin - Sun Exposed (Lower leg)_GTExTPM', 'Prostate_GTExTPM', 'Skin - Not Sun Exposed (Suprapubic)_GTExTPM', 'Esophagus - Muscularis_GTExTPM', 'Small Intestine - Terminal Ileum_GTExTPM', 'Spleen_GTExTPM', 'Stomach_GTExTPM', 'Thyroid_GTExTPM', 'Uterus_GTExTPM', 'Whole Blood_GTExTPM', 'Pancreas_GTExTPM', 'Esophagus - Mucosa_GTExTPM', 'Vagina_GTExTPM', 'Colon - Transverse_GTExTPM', 'Adipose - Subcutaneous_GTExTPM', 'Esophagus - Gastroesophageal Junction_GTExTPM', 'Artery - Aorta_GTExTPM', 'Artery - Coronary_GTExTPM', 'Artery - Tibial_GTExTPM', 'Bladder_GTExTPM', 'Brain - Anterior cingulate cortex (BA24)_GTExTPM', 'Brain - Caudate (basal ganglia)_GTExTPM', 'Brain - Cerebellar Hemisphere_GTExTPM', 'Brain - Cerebellum_GT

  0%|          | 0/200 [00:00<?, ?it/s]

12 attributes confirmed important: ['HIPred', 'mousescore_Exomiser', 'IPA_BP_annotation', 'Brain - Amygdala_GTExTPM', 'Fallopian Tube_GTExTPM', 'SDI', 'Kidney - Cortex_GTExTPM', 'Heart - Atrial Appendage_GTExTPM', 'Liver_GTExTPM', 'pLI_ExAC', 'Cells - EBV-transformed lymphocytes_GTExTPM', 'Testis_GTExTPM']
8 attributes confirmed unimportant: ['wgEncodeBroadHmmHuvecHMM.count', 'fishscore_Exomiser', 'Pituitary_GTExTPM', 'IPA_Activity', 'GDI_Score', 'betamax', 'humanscore_Exomiser', 'Adrenal Gland_GTExTPM']
0 tentative attributes remains: []


,HIPred,mousescore_Exomiser,IPA_BP_annotation,Brain - Amygdala_GTExTPM,Fallopian Tube_GTExTPM,SDI,Kidney - Cortex_GTExTPM,Heart - Atrial Appendage_GTExTPM,Liver_GTExTPM,pLI_ExAC,Cells - EBV-transformed lymphocytes_GTExTPM,Testis_GTExTPM
0,0.670620,0.884429,1.0,1.918180,18.294000,4.777847,1.054690,7.486480,8.690460,0.000701,0.039309,0.506314
1,0.544935,0.003954,0.0,37.922680,347.827309,6.822910,85.083259,1203.937546,82.508557,0.351865,77.903175,79.111953
2,0.668119,0.012444,0.0,46.094020,71.058651,14.328770,63.746061,66.441105,69.673243,0.653296,70.863917,55.958586
3,0.609159,0.039973,0.0,30.564139,73.453239,6.120033,62.967336,73.533747,85.692167,0.375502,56.495594,63.618441
4,0.645000,0.007907,0.0,35.495538,70.393748,6.614234,60.942632,71.716759,73.865410,0.624623,60.652245,63.035546
...,...,...,...,...,...,...,...,...,...,...,...,...
288,0.745583,0.000000,0.0,11.502500,18.552000,7.929407,12.615300,7.934430,12.703200,0.046392,12.181300,24.890700
289,0.645917,0.013511,0.0,37.599341,70.753121,6.381988,65.038186,78.647709,87.669337,0.564259,76.340758,67.574890
290,0.792615,0.000000,0.0,0.635257,6.764240,6.843602,2.180240,1.403160,3.386490,0.272667,1.596780,6.498470
291,0.660858,0.026918,0.0,34.087908,68.245929,6.628453,57.417707,67.161857,75.552637,0.552313,65.798952,65.294446
